In [53]:
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score

np.random.seed = 24

In [21]:
df = pd.read_hdf('data/joined.h5')
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_name_eng,item_category_name,item_category_name_eng,shop_name,shop_name_eng,revenue_day,return
0,2013-01-02,0,59,22154,999.0,1.0,ЯВЛЕНИЕ 2012 (BD),37,ЯВЛЕНИЕ 2012 (BD)-unable to translate,Кино - Blu-Ray,Cinema - Blu-Ray,"Ярославль ТЦ ""Альтаир""","Yaroslavl Shopping center ""Altair""",999.0,False
1,2013-01-03,0,25,2552,899.0,1.0,DEEP PURPLE The House Of Blue Light LP,58,DEEP PURPLE The House Of Blue Light LP,Музыка - Винил,Music - Vinyl,"Москва ТРК ""Атриум""","Moscow TRK ""Atrium""",899.0,False


In [22]:
modeling = df.groupby(['date_block_num','shop_id','item_category_id','item_id']).sum().reset_index()
modeling.head()
modeling['single_item_price'] = modeling.item_price/modeling.item_cnt_day

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,revenue_day,return
0,0,0,2,5572,7932.0,10.0,13220.0,0.0
1,0,0,2,5573,560.0,1.0,560.0,0.0
2,0,0,2,5575,2418.0,4.0,3224.0,0.0
3,0,0,2,5576,11155.0,5.0,11155.0,0.0
4,0,0,2,5609,2381.0,1.0,2381.0,0.0


In [64]:
# We need to predict item_cnt_day for Nov 2015 which would be dateblock 34
# Just doing a baseline from a random split to start off with
# going to aggregate the data at a dateblock/item level and just use the data i'm given
pipeline = Pipeline([
    #('scaler',preprocessing.StandardScaler()),
    ('rf', DecisionTreeRegressor())
])
print(pipeline.get_params)

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('rf', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'))])>


In [61]:
cat_cols = ['shop_id','item_id','item_category_id']
cont_cols = ['date_block_num','single_item_price']
prediction_col = ['item_cnt_day']

enc = preprocessing.LabelEncoder()
for c in cat_cols:
    modeling[c] = enc.fit_transform(modeling[c])
    
print(modeling.head(3))

X = modeling[feature_cols].values
y = modeling[prediction_col].values

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3)

print('X_train shape:\t',X_train.shape)
print('y_train shape:\t',y_train.shape)
print('X_test shape:\t',X_test.shape)
print('y_test shape:\t',y_test.shape)

   date_block_num  shop_id  item_category_id  item_id  item_price  \
0               0        0                12    16986      7932.0   
1               0        0                12    16987       560.0   
2               0        0                12    16989      2418.0   

   item_cnt_day  revenue_day  return  
0          10.0      13220.0     0.0  
1           1.0        560.0     0.0  
2           4.0       3224.0     0.0  
X_train shape:	 (1126386, 5)
y_train shape:	 (1126386, 1)
X_test shape:	 (482738, 5)
y_test shape:	 (482738, 1)


In [65]:
pipeline.fit(X_train,y_train)

y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

print('R2 Training:\t',r2_score(y_train, y_train_pred))
print('R2 Test:\t',r2_score(y_test, y_test_pred))
print('MSE Training:\t',mean_squared_error(y_train, y_train_pred))
print('MSE Test:\t',mean_squared_error(y_test, y_test_pred))

R2 Training:	 1.0
R2 Test:	 0.499171145095
MSE Training:	 0.0
MSE Test:	 40.4229561377
